# Importações e funções

In [ ]:
import pandas as pd
import socket
import requests
import dns.resolver
import time

# Função para obter o IP de um domínio
def get_ip(domain):
    try:
        return socket.gethostbyname(domain)
    except Exception as e:
        return None

# Função para obter a localização do IP usando a API ipinfo.io
def get_location(ip):
    if ip:
        try:
            response = requests.get(f'https://ipinfo.io/{ip}/json')
            data = response.json()
            loc = data.get('loc', '').split(',')
            latitude = loc[0] if len(loc) > 0 else ''
            longitude = loc[1] if len(loc) > 1 else ''
            return {
                'city': data.get('city', ''),
                'region': data.get('region', ''),
                'country': data.get('country', ''),
                'org': data.get('org', ''),
                'postal': data.get('postal', ''),
                'timezone': data.get('timezone', ''),
                'hostname': data.get('hostname', ''),
                'latitude': latitude,
                'longitude': longitude,
            }
        except Exception as e:
            return {
                'city': '',
                'region': '',
                'country': '',
                'org': '',
                'postal': '',
                'timezone': '',
                'hostname': '',
                'latitude': '',
                'longitude': '',
            }
    return {
        'city': '',
        'region': '',
        'country': '',
        'org': '',
        'postal': '',
        'timezone': '',
        'hostname': '',
        'latitude': '',
        'longitude': '',
    }

# Função para obter o DNS reverso de um IP
def get_reverse_dns(ip):
    if ip:
        try:
            return socket.gethostbyaddr(ip)[0]
        except Exception as e:
            return None
    return None

# Função para obter o número de registros DNS de um IP
def get_dns_count(ip):
    if ip:
        try:
            answers = dns.resolver.resolve(ip, 'A')
            return len(answers)
        except Exception as e:
            return 0
    return 0






# Ler dataset

In [ ]:
'''
#Ja corridos: 
#dataset_block6
'''
# Carregar o dataset
df = pd.read_csv('Datasets/dataset2.csv', delimiter=';')


# Linhas em que a label não se encontra preenchida

In [ ]:
# Substituir valores vazios na coluna "label" por -1
df['label'].fillna(-1, inplace=True)

# Contar os valores únicos na coluna "label"
label_counts = df['label'].value_counts()
print(label_counts)

# Encontrar os índices das linhas com valor -1 na coluna "label"
indices_to_drop = df[df['label'] == -1].index

# Remover as linhas com valor -1 na coluna "label"
df.drop(indices_to_drop, inplace=True)

# Reverse module DNS 2

In [ ]:
# Adicionar colunas para armazenar IP, DNS, Reverse DNS e número de DNS
df['IP'] = df['URL'].apply(lambda url: get_ip(url.split('//')[1]))
df['DNS'] = df['URL'].apply(lambda url: url.split('//')[1])
df['Reverse DNS'] = df['IP'].apply(get_reverse_dns)
df['n_dns'] = df['IP'].apply(get_dns_count)

# Adicionar contador para pausar a cada 100 registros
for i in range(0, len(df), 100):
    if i > 0 and i % 100 == 0:
        print(f"Processed {i} records. Sleeping for 30 seconds...")
        time.sleep(30)

    # Processar blocos de 100 registros
    chunk = df.iloc[i:i+100]
    
    # Obter informações de localização e adicionar colunas separadas
    location_data = chunk['IP'].apply(get_location)
    df.loc[i:i+100, 'City'] = location_data.apply(lambda loc: loc['city'])
    df.loc[i:i+100, 'Region'] = location_data.apply(lambda loc: loc['region'])
    df.loc[i:i+100, 'Country'] = location_data.apply(lambda loc: loc['country'])
    df.loc[i:i+100, 'Org'] = location_data.apply(lambda loc: loc['org'])
    df.loc[i:i+100, 'Postal'] = location_data.apply(lambda loc: loc['postal'])
    df.loc[i:i+100, 'Timezone'] = location_data.apply(lambda loc: loc['timezone'])
    df.loc[i:i+100, 'Hostname'] = location_data.apply(lambda loc: loc['hostname'])
    df.loc[i:i+100, 'Latitude'] = location_data.apply(lambda loc: loc['latitude'])
    df.loc[i:i+100, 'Longitude'] = location_data.apply(lambda loc: loc['longitude'])

    # Removing unnamed columns using drop function
df.drop(df.columns[df.columns.str.contains('Unnamed', case=False)], axis=1, inplace=True)

# Salvar o resultado em um novo arquivo CSV
df.to_csv('dataset2_reverse_dns.csv', index=False,sep=';')

# Exibir o DataFrame
print(df.head())

In [ ]:
""" import pandas as pd
from sklearn.model_selection import train_test_split

# Carregar o dataset
df = pd.read_csv('PhishingDataset.csv', delimiter='')

# Converter todas as colunas para strings
df = df.astype(str)

# Dividir o dataset em dois novos datasets (50% e 50%)
df1, df2 = train_test_split(df, test_size=0.5, random_state=42)

# Salvar os dois novos datasets em arquivos CSV separados
df1.to_csv('dataset1.csv', index=False,sep=';')
df2.to_csv('dataset2.csv', index=False,sep=';') """


In [ ]:
""" import pandas as pd

# Carregar o dataset com delimitador ','
df = pd.read_csv('total.csv', delimiter=',')

# Salvar o dataset com delimitador ';'
df.to_csv('total_semicolon.csv', index=False, sep=';') """

""" import pandas as pd

# Carregar o dataset com delimitador ';'
df = pd.read_csv('Datasets/dataset1.csv', delimiter=';')

# Calcular o número de linhas em 20% do dataset
n = len(df)
block_size = int(n * 0.2)

# Shuffle the dataset
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Create a list to hold each 20% block
blocks = []

# Generate the 20% blocks
for i in range(5):
    start_index = i * block_size
    end_index = (i + 1) * block_size
    block = df_shuffled.iloc[start_index:end_index]
    blocks.append(block)
    # Save each block to a new CSV file
    block.to_csv(f'Datasets/dataset_block_{i+1}.csv', index=False, sep=';')

# In case the dataset size is not perfectly divisible by 5, add the remaining rows to the last block
if end_index < n:
    remaining_block = df_shuffled.iloc[end_index:]
    remaining_block.to_csv(f'Datasets/dataset_block_6.csv', index=False, sep=';') """

